In [25]:
import sys, os, getopt, re
from functools import wraps
from glob import glob
import pickle
import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd

# Latin Data Splits and Verb Suffixing Corrections
This project was broken up into two main parts: Creating the data splits, and editing the baseline code to more accurately predict the morphological inflections of the 什么什么什么

## A. Splitting the Data
Since Latin had no splits, our group needed to manually make the train, test, and dev sets. This task alone has a lot of factors to consider to avoid artificially inflating the accuracy of our code. Additionally, we started with nearly one million lines of data, which needed to be cut down

There are many factors to keep in mind while splitting the data. Below are the two main factors we focused on:

#### Size of the data

The size of the training data correlates with the accuracy of the code (Kodner et al. 2023). By cutting down the data, we save both on artificially raised accuracy and runtime

#### Splitting-by-form vs. Splitting-by-lemma


In [ ]:
def hamming(s,t):
    return sum(1 for x,y in zip(s,t) if x != y)a

In [ ]:
def halign(s,t):
    """Align two strings by Hamming distance."""
    slen = len(s)
    tlen = len(t)
    minscore = len(s) + len(t) + 1
    for upad in range(0, len(t)+1):
        upper = '_' * upad + s + (len(t) - upad) * '_'
        lower = len(s) * '_' + t
        score = hamming(upper, lower)
        if score < minscore:
            bu = upper
            bl = lower
            minscore = score

    for lpad in range(0, len(s)+1):
        upper = len(t) * '_' + s
        lower = (len(s) - lpad) * '_' + t + '_' * lpad
        score = hamming(upper, lower)
        if score < minscore:
            bu = upper
            bl = lower
            minscore = score

    zipped = list(zip(bu,bl))
    newin  = ''.join(i for i,o in zipped if i != '_' or o != '_')
    newout = ''.join(o for i,o in zipped if i != '_' or o != '_')
    return newin, newout

In [13]:
def levenshtein(s, t, inscost = 1.0, delcost = 1.0, substcost = 1.0):
    """Recursive implementation of Levenshtein, with alignments returned."""
    @memolrec
    def lrec(spast, tpast, srem, trem, cost):
        if len(srem) == 0:
            return spast + len(trem) * '_', tpast + trem, '', '', cost + len(trem)
        if len(trem) == 0:
            return spast + srem, tpast + len(srem) * '_', '', '', cost + len(srem)

        addcost = 0
        if srem[0] != trem[0]:
            addcost = substcost

        return min((lrec(spast + srem[0], tpast + trem[0], srem[1:], trem[1:], cost + addcost),
                   lrec(spast + '_', tpast + trem[0], srem, trem[1:], cost + inscost),
                   lrec(spast + srem[0], tpast + '_', srem[1:], trem, cost + delcost)),
                   key = lambda x: x[4])

    answer = lrec('', '', s, t, 0)
    return answer[0],answer[1],answer[4]

In [15]:
def memolrec(func):
    """Memoizer for Levenshtein."""
    cache = {}
    @wraps(func)
    def wrap(sp, tp, sr, tr, cost):
        if (sr,tr) not in cache:
            res = func(sp, tp, sr, tr, cost)
            cache[(sr,tr)] = (res[0][len(sp):], res[1][len(tp):], res[4] - cost)
        return sp + cache[(sr,tr)][0], tp + cache[(sr,tr)][1], '', '', cost + cache[(sr,tr)][2]
    return wrap

In [17]:
def alignprs(lemma, form):
    """Break lemma/form into three parts:
    IN:  1 | 2 | 3
    OUT: 4 | 5 | 6
    1/4 are assumed to be prefixes, 2/5 the stem, and 3/6 a suffix.
    1/4 and 3/6 may be empty.
    """

    al = levenshtein(lemma, form, substcost = 1.1) # Force preference of 0:x or x:0 by 1.1 cost
    alemma, aform = al[0], al[1]
    # leading spaces
    lspace = max(len(alemma) - len(alemma.lstrip('_')), len(aform) - len(aform.lstrip('_')))
    # trailing spaces
    tspace = max(len(alemma[::-1]) - len(alemma[::-1].lstrip('_')), len(aform[::-1]) - len(aform[::-1].lstrip('_')))
    return alemma[0:lspace], alemma[lspace:len(alemma)-tspace], alemma[len(alemma)-tspace:], aform[0:lspace], aform[lspace:len(alemma)-tspace], aform[len(alemma)-tspace:]


In [19]:
def prefix_suffix_rules_get(lemma, form):
    """Extract a number of suffix-change and prefix-change rules
    based on a given example lemma+inflected form."""
    lp,lr,ls,fp,fr,fs = alignprs(lemma, form) # Get six parts, three for in three for out

    # Suffix rules
    ins  = lr + ls + ">"
    outs = fr + fs + ">"
    srules = set()
    for i in range(min(len(ins), len(outs))):
        srules.add((ins[i:], outs[i:]))
    srules = {(x[0].replace('_',''), x[1].replace('_','')) for x in srules}

    # Prefix rules
    prules = set()
    if len(lp) >= 0 or len(fp) >= 0:
        inp = "<" + lp
        outp = "<" + fp
        for i in range(0,len(fr)):
            prules.add((inp + fr[:i],outp + fr[:i]))
            prules = {(x[0].replace('_',''), x[1].replace('_','')) for x in prules}

    return prules, srules

In [21]:
def apply_best_rule(lemma, msd, allprules, allsrules):
    """Applies the longest-matching suffix-changing rule given an input
    form and the MSD. Length ties in suffix rules are broken by frequency.
    For prefix-changing rules, only the most frequent rule is chosen."""

    bestrulelen = 0
    base = "<" + lemma + ">"
    if msd not in allprules and msd not in allsrules:
        return lemma # Haven't seen this inflection, so bail out

    if msd in allsrules:
        applicablerules = [(x[0],x[1],y) for x,y in allsrules[msd].items() if x[0] in base]
        if applicablerules:
            bestrule = max(applicablerules, key = lambda x: (len(x[0]), x[2], len(x[1])))
            base = base.replace(bestrule[0], bestrule[1])

    if msd in allprules:
        applicablerules = [(x[0],x[1],y) for x,y in allprules[msd].items() if x[0] in base]
        if applicablerules:
            bestrule = max(applicablerules, key = lambda x: (x[2]))
            base = base.replace(bestrule[0], bestrule[1])

    base = base.replace('<', '')
    base = base.replace('>', '')
    return base

In [23]:
def numleadingsyms(s, symbol):
    return len(s) - len(s.lstrip(symbol))


def numtrailingsyms(s, symbol):
    return len(s) - len(s.rstrip(symbol))
    

In [ ]:
# Get the input file as an array so I can skip over entries with +s
lines = []
with open('Latin_stuff/ORIGINAL_lat.trn') as f:
    lines = [line.rstrip('\n') for line in f]
    
# Read in data as dataframe
lat = pd.read_table("Latin_stuff/ORIGINAL_lat.trn", sep='\t', names=['Lemon', 'Infection', 'Infected'], skiprows=lambda x: '+' in lines[x])
print(lat.shape)

In [ ]:
# get a list of the unique lemmas and infinitives
infinitives = lat[lat['Infection'] == 'V;NFIN;ACT;PRS']
uniqueLemmas = lat.drop_duplicates(subset = ['Lemon', 'PartoSpeech'])

In [ ]:
# Split the unique lemmas into dataframes by part of speech while also cutting it down using numbers I calculated elsewhere
partSample = uniqueLemmas[uniqueLemmas['PartoSpeech'] == 'V.PTCP'].sample(n=112)
adjSample = uniqueLemmas[uniqueLemmas['PartoSpeech'] == 'ADJ'].sample(n=105)
nounSample = uniqueLemmas[uniqueLemmas['PartoSpeech'] == 'N'].sample(n=185)
verbSample = uniqueLemmas[uniqueLemmas['PartoSpeech'] == 'V'].sample(n=41)
propSample = uniqueLemmas[uniqueLemmas['PartoSpeech'] == 'PROPN'].sample(n=343)

In [ ]:
# Split the dataframes randomly into train, test, and dev sets in a 10:1:1 ratio
partTrain, partTest = train_test_split(partSample, test_size=2000/12000, random_state=56)
partTest, partDev = train_test_split(partTest, test_size=0.5, random_state=56)

adjTrain, adjTest = train_test_split(adjSample, test_size=2000/12000, random_state=56)
adjTest, adjDev = train_test_split(adjTest, test_size=0.5, random_state=56)

nounTrain, nounTest = train_test_split(nounSample, test_size=2000/12000, random_state=56)
nounTest, nounDev = train_test_split(nounTest, test_size=0.5, random_state=56)

verbTrain, verbTest = train_test_split(verbSample, test_size=2000/12000, random_state=56)
verbTest, verbDev = train_test_split(verbTest, test_size=0.5, random_state=56)

propTrain, propTest = train_test_split(propSample, test_size=2000/12000, random_state=56)
propTest, propDev = train_test_split(propTest, test_size=0.5, random_state=56)

In [ ]:
# Concatenate the dataframes together to get all the unique lemmas in the test, train, and dev sets
uniqueTrain = pd.concat([partTrain, adjTrain, nounTrain, verbTrain, propTrain])
uniqueTest = pd.concat([partTest, adjTest, nounTest, verbTest, propTest])
uniqueDev = pd.concat([partDev, adjDev, nounDev, verbDev, propDev])

In [ ]:
# Method to get all of the other lemmas which match with the lemmas in the unique list and return it as a dataframe
def getlist(lemons):
    splitslist = []
    for lemon in lemons['Lemon']:
        for row in lat[lat['Lemon'] == lemon].to_numpy().tolist():
            splitslist.append(row)
    return pd.DataFrame(splitslist, columns= ["Lemon", "Infected", "Infection", "PartoSpeech"])

# Call the method to get the full train, test, and dev sets
train = getlist(uniqueTrain)
test = getlist(uniqueTest)
dev = getlist(uniqueDev)

In [ ]:
# Print the sizes of the train, test, and dev sets to check that they look right
print("Train set size", train.shape)
print("Test set size", test.shape)
print("Dev set size", dev.shape)

# Print the number of rows per part of speech in the training set to check that they look right
print("Train set split into parts of speech")
print(train[train['PartoSpeech'] == 'V.PTCP'].shape)
print(train[train['PartoSpeech'] == 'ADJ'].shape)
print(train[train['PartoSpeech'] == 'N'].shape)
print(train[train['PartoSpeech'] == 'V'].shape)
print(train[train['PartoSpeech'] == 'PROPN'].shape)

# Print the number of rows per part of speech in the test set to check that they look right
print("Test set split into parts of speech")
print(test[test['PartoSpeech'] == 'V.PTCP'].shape)
print(test[test['PartoSpeech'] == 'ADJ'].shape)
print(test[test['PartoSpeech'] == 'N'].shape)
print(test[test['PartoSpeech'] == 'V'].shape)
print(test[test['PartoSpeech'] == 'PROPN'].shape)

# Print the number of rows per part of speech in the dev set to check that they look right
print("Dev set split into parts of speech")
print(dev[dev['PartoSpeech'] == 'V.PTCP'].shape)
print(dev[dev['PartoSpeech'] == 'ADJ'].shape)
print(dev[dev['PartoSpeech'] == 'N'].shape)
print(dev[dev['PartoSpeech'] == 'V'].shape)
print(dev[dev['PartoSpeech'] == 'PROPN'].shape)

In [ ]:
# Write a method to convert the dataframes to files based on Emily's code
def toFile(frame, fileName, fileType):
    frame.to_csv(path_or_buf= './Latin_stuff/' + fileName + fileType,sep= "\t", encoding= "utf8", index= False, header=False, columns= ["Lemon", "Infected", "Infection"])

# Convert the test, train, and dev sets to files
toFile(train, 'lat', '.trn')
toFile(test, 'lat', '.tst')
toFile(dev, 'lat', '.dev')